## Saving a Cats v Dogs Model

This is a minimal example showing how to train a fastai model on Kaggle, and save it so you can use it in your app.

In [ ]:
# Make sure we've got the latest version of fastai:
!pip install -Uqq fastai

First, import all the stuff we need from fastai:

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *  #Needed for ImageClassiferCleaner

Download and decompress our dataset, which is pictures of dogs and cats:

In [ ]:
path = untar_data(URLs.PETS)/'images'
print(path)

fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink)


We need a way to label our images as dogs or cats. In this dataset, pictures of cats are given a filename that starts with a capital letter:

In [ ]:
# The input to this function is a string - the name of the file.
def is_cat(x): 
    if (x[0].isupper()):
        return "Cat"
    else:
        return "Dog"

Now we can create our `DataLoaders`:

In [ ]:
# Why is a string passed to is_cat function vs. a Path
dls = ImageDataLoaders.from_name_func('.',
    get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192))

dls.valid.show_batch(max_n=4, nrows=1)

... and train our model, a resnet18 (to keep it small and fast):

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()


In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner 

Note:  I need to recrate the data loader. 

See - https://forums.fast.ai/t/how-to-retrain-the-bear-classifier-model-after-cleaning-the-data/86231

In [ ]:
for idx in cleaner.delete(): 
    cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): 
    shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
dls = ImageDataLoaders.from_name_func('.',
    get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192))
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Now we can export our trained `Learner`. This contains all the information needed to run the model:

In [ ]:
learn.export('model.pkl')

Finally, open the Kaggle sidebar on the right if it's not already, and find the section marked "Output". Open the `/kaggle/working` folder, and you'll see `model.pkl`. Click on it, then click on the menu on the right that appears, and choose "Download". After a few seconds, your model will be downloaded to your computer, where you can then create your app that uses the model.